OVERVIEW - this Notebook will be used to complete the IBM Data Science Certificate Capstone Project. Project will include 3 sections as follows (i) Scraping Data from a Wikipedia Page (ii) Establishing a dataframe which includes location data for neigborhoods such as latitudes and longitudes  (iii) performing clustering/segmentation of the neighborhood and creating map visualizations

In [6]:
# importing initial python libraries
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# PART I - SCRAPING DATA FROM WIKIPEDIA PAGE

In [1]:
#installing necessary packages and the Requests Library. Using BeautifulSoup to scrape the wikipage
!pip install bs4
import requests
from bs4 import BeautifulSoup

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=e2bb568c3a24ee3d8b760f0f83eaee855c273dcf0f375aeb60f6aee9a25d93e1
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


The wiki page 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' contains list of postal codes. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

In [2]:
#downloading source code for wiki page and creating the soup object
url= 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.content)
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

Parsing through the web page code and extracting the relevant information which is contained in the class attributes ('td'), skipping rows that do not have assigned postalcodes

In [16]:

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

df.shape

(103, 3)

In [14]:
#cleaning up some data for boroughs that have unusual names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [15]:
df.tail(20)

,PostalCode,Borough,Neighborhood
83,M4T,Central Toronto,"Moore Park, Summerhill East"
84,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
87,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
90,M1W,Scarborough,"Steeles West, L'Amoreaux West"
91,M4W,Downtown Toronto,Rosedale
92,M5W,Downtown Toronto Stn A,Enclave of M5E


In [17]:
#size of dataframe
df.shape

(103, 3)